In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from global_import import *
from valuation.val import perform_valuation
from fm import fama_macbeth_reg_panel_nw

In [2]:
ret_decomp_dcf5 = \
          pd.read_csv(f'{DATA_FOLDER}/decomp_dcf5y_RF_rolling_1980_2019.csv', index_col=0).replace([np.inf, -np.inf], np.nan)
ret_decomp_dcf5 = ret_decomp_dcf5[ret_decomp_dcf5.prc>5]
ret_decomp_dcf5['jdate'] = pd.to_datetime(ret_decomp_dcf5.jdate)
ret_decomp_dcf5['bm'] = np.log(ret_decomp_dcf5['be']) - ret_decomp_dcf5['me']

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
ret_decomp_dcf5.head(3)

,permno,ticker,exchcd,year,jdate,exchcd.1,vt,me,be,prc,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_48,logret_60,cumret_12,cumret_36,cumret_48,cumret_60,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+48,me+48,be+48,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+48,r*_t+48,r_unchg+48,r_chg+48,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp,bm
203,10001,EWST,3,2008,2008-12-31,3,12.899791,10.477097,25293.0,8.26,0.004304,0.070154,0.352370,0.004313,0.072673,0.422435,0.280987,0.482664,0.332560,0.237488,0.324436,0.620385,0.394534,0.268060,NaN,10.468062,27152.0,NaN,10.518353,27152.0,NaN,10.712601,27152.0,NaN,11.319507,30322.0,NaN,11.441630,35907.0,NaN,11.239867,73898.0,NaN,11.337733,77856.0,0.004304,NaN,0.060567,NaN,0.070154,NaN,0.060567,NaN,0.280987,NaN,0.060567,NaN,0.352370,NaN,0.060567,NaN,0.482664,NaN,0.060567,NaN,0.332560,NaN,0.060567,NaN,0.237488,NaN,0.060567,NaN,-2.422694,-0.338814
202,10001,EWST,3,2008,2008-11-30,3,12.899791,10.349001,25293.0,7.18,0.233225,0.195851,0.446356,0.262666,0.216345,0.562608,0.275266,0.577084,0.492865,0.443105,0.316881,0.780838,0.636999,0.557536,NaN,10.556208,27152.0,NaN,10.516561,27152.0,NaN,10.566283,27152.0,NaN,11.272806,30322.0,NaN,11.395027,35907.0,NaN,11.260027,73898.0,NaN,11.403983,77856.0,0.233225,NaN,0.063770,NaN,0.195851,NaN,0.063770,NaN,0.275266,NaN,0.063770,NaN,0.446356,NaN,0.063770,NaN,0.577084,NaN,0.063770,NaN,0.492865,NaN,0.063770,NaN,0.443105,NaN,0.063770,NaN,-2.550790,-0.210718
201,10001,EWST,3,2008,2008-10-31,3,12.899791,10.493957,25293.0,8.30,0.039126,0.053513,0.433014,0.039902,0.054971,0.541898,0.128796,0.442696,0.400036,0.448620,0.137458,0.556899,0.491879,0.566150,NaN,10.505974,27152.0,NaN,10.506436,27152.0,NaN,10.550869,27152.0,NaN,11.148880,30322.0,NaN,11.403914,35907.0,NaN,11.311315,73898.0,NaN,11.546446,77856.0,0.039126,NaN,0.060146,NaN,0.053513,NaN,0.060146,NaN,0.128796,NaN,0.060146,NaN,0.433014,NaN,0.060146,NaN,0.442696,NaN,0.060146,NaN,0.400036,NaN,0.060146,NaN,0.448620,NaN,0.060146,NaN,-2.405834,-0.355674


In [4]:
ret_decomp_dcf5_nyse = ret_decomp_dcf5[ret_decomp_dcf5.exchcd==1]

In [5]:
ret_decomp_dcf5_nyse.groupby(pd.qcut(ret_decomp_dcf5_nyse.me, 10)).agg('mean')[['misp', 'r_t+36']]

,misp,r_t+36
me,,
"(6.763999999999999, 11.56]",-0.770790,0.099255
"(11.56, 12.274]",-0.285658,0.091476
"(12.274, 12.851]",-0.217776,0.081247
"(12.851, 13.359]",-0.119797,0.072323
"(13.359, 13.827]",-0.007294,0.088176
"(13.827, 14.292]",0.012854,0.082069
"(14.292, 14.787]",-0.008050,0.078264
"(14.787, 15.363]",-0.027583,0.069640
"(15.363, 16.226]",-0.066345,0.070700


In [6]:
ret_decomp_dcf5_nyse.groupby(pd.qcut(ret_decomp_dcf5_nyse.me, 10)).agg('std')[['misp', 'r_t+36']]

,misp,r_t+36
me,,
"(6.763999999999999, 11.56]",1.733375,0.772598
"(11.56, 12.274]",1.471194,0.740391
"(12.274, 12.851]",1.473130,0.754937
"(12.851, 13.359]",1.423078,0.762697
"(13.359, 13.827]",1.352782,0.754540
"(13.827, 14.292]",1.340089,0.706918
"(14.292, 14.787]",1.326548,0.680530
"(14.787, 15.363]",1.278511,0.639000
"(15.363, 16.226]",1.246747,0.589675


In [7]:
ret_decomp_dcf5_nyse.groupby(pd.qcut(ret_decomp_dcf5_nyse.bm, 10)).agg('mean')[['misp', 'r_t+36']]

,misp,r_t+36
bm,,
"(-9.66, -1.664]",0.752362,-0.022393
"(-1.664, -1.255]",0.492536,0.020460
"(-1.255, -0.994]",0.310145,0.069077
"(-0.994, -0.771]",0.195814,0.080292
"(-0.771, -0.561]",0.081374,0.082782
"(-0.561, -0.358]",-0.031078,0.085538
"(-0.358, -0.148]",-0.151311,0.112185
"(-0.148, 0.0928]",-0.246832,0.137131
"(0.0928, 0.49]",-0.386380,0.165853


In [8]:
ret_decomp_dcf5_nyse.groupby(pd.qcut(ret_decomp_dcf5_nyse.bm, 10)).agg('std')[['misp', 'r_t+36']]

,misp,r_t+36
bm,,
"(-9.66, -1.664]",1.283384,0.710600
"(-1.664, -1.255]",1.045659,0.670923
"(-1.255, -0.994]",1.005158,0.642898
"(-0.994, -0.771]",0.988934,0.678758
"(-0.771, -0.561]",0.972753,0.685396
"(-0.561, -0.358]",0.977191,0.696421
"(-0.358, -0.148]",0.994416,0.696721
"(-0.148, 0.0928]",0.955956,0.713419
"(0.0928, 0.49]",0.938325,0.788708


In [9]:
regdf = ret_decomp_dcf5[['jdate', 'permno', 'ticker', 'me', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()
print(fama_macbeth_reg_panel_nw(regdf, xname='me', yname='misp'))


           count      mean  std_error      tstat  std_error_nw   tstat_nw
alpha_i_t  468.0 -4.920675   0.060306 -81.595782      0.145431 -33.835051
me         468.0  0.354930   0.003553  99.901915      0.008508  41.715226


In [10]:
regdf = ret_decomp_dcf5[['jdate', 'permno', 'ticker', 'bm', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()
print(fama_macbeth_reg_panel_nw(regdf, xname='bm', yname='misp'))


           count      mean  std_error       tstat  std_error_nw   tstat_nw
alpha_i_t  468.0 -0.719149   0.024164  -29.761476      0.058323 -12.330462
bm         468.0 -0.744402   0.003761 -197.911716      0.009029 -82.448447


In [11]:
ret_decomp_dcf5 = ret_decomp_dcf5[['jdate', 'permno', 'ticker', 'bm', 'me', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()

In [12]:
ret_decomp_dcf5_annual = ret_decomp_dcf5[ret_decomp_dcf5.jdate.dt.month==6]

In [13]:
%matplotlib notebook

In [14]:
import matplotlib.cm as cm
# Calculate the point density
x=ret_decomp_dcf5_annual['me']
y=ret_decomp_dcf5_annual['misp']
sns.jointplot(x, y ,color='blue', kind="reg",joint_kws = {'scatter_kws':dict(alpha=0.02),
                                                         'line_kws':{'color':'cyan'}})
plt.show()

<IPython.core.display.Javascript object>

In [15]:
import matplotlib.cm as cm
# Calculate the point density
x=ret_decomp_dcf5_annual['bm']
y=ret_decomp_dcf5_annual['misp']
sns.jointplot(x, y ,color='blue', kind="reg",joint_kws = {'scatter_kws':dict(alpha=0.02),
                                                         'line_kws':{'color':'cyan'}})
plt.show()

<IPython.core.display.Javascript object>

In [16]:
ret_decomp_dcf5[['jdate', 'permno', 'ticker', 'bm', 'me', 'misp']].describe()

,permno,bm,me,misp
count,1.148779e+06,1.148779e+06,1.148779e+06,1.148779e+06
mean,5.550085e+04,-6.290988e-01,1.310880e+01,-2.140558e-01
std,2.611250e+04,1.139468e+00,2.020718e+00,1.473774e+00
min,1.000100e+04,-9.658938e+00,2.617396e+00,-1.671444e+01
25%,3.166800e+04,-1.206724e+00,1.165273e+01,-8.615113e-01
50%,5.920500e+04,-6.177603e-01,1.306979e+01,-1.008347e-02
75%,7.930700e+04,-9.395413e-02,1.448257e+01,6.852850e-01
max,9.343600e+04,9.757427e+00,2.098929e+01,8.235739e+00
